# HW: X-ray images classification
--------------------------------------

Before you begin, open Mobaxterm and connect to triton with the user and password you were give with. Activate the environment `2ndPaper` and then type the command `pip install scikit-image`.

In this assignment you will be dealing with classification of 32X32 X-ray images of the chest. The image can be classified into one of four options: lungs (l), clavicles (c), and heart (h) and background (b). Even though those labels are dependent, we will treat this task as multiclass and not as multilabel. The dataset for this assignment is located on a shared folder on triton (`/MLdata/MLcourse/X_ray/'`).

In [1]:
import os
import numpy as np
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Dropout
from tensorflow.keras.layers import Flatten, InputLayer
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import *

from tensorflow.keras.initializers import Constant
from tensorflow.keras.datasets import fashion_mnist
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from skimage.io import imread

from skimage.transform import rescale, resize, downscale_local_mean
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options =
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [3]:
def preprocess(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        src = imread(os.path.join(datapath, fn),1)
        img = resize(src,(32,32),order = 3)
        
        images[ii,:,:,0] = img
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    BaseImages = images
    BaseY = Y
    return BaseImages, BaseY

In [4]:
def preprocess_train_and_val(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        images[ii,:,:,0] = imread(os.path.join(datapath, fn),1)
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    return images, Y

In [5]:
#Loading the data for training and validation:
#src_data = '/MLdata/MLcourse/X_ray/'
src_data = 'C:\\Users\\ofeka\\Desktop\\Projects\\Machine Learning\\Homework\\HW4\\X_ray\\'
train_path = src_data + 'train'
val_path = src_data + 'validation'
test_path = src_data + 'test'
BaseX_train , BaseY_train = preprocess_train_and_val(train_path)
BaseX_val , BaseY_val = preprocess_train_and_val(val_path)
X_test, Y_test = preprocess(test_path)

In [6]:
keras.backend.clear_session()

In [7]:
# BLOCK FOR TESTING STUFF:

print(BaseX_train.shape)
print(BaseX_train.flatten().shape)
print(BaseY_train.shape)
print(BaseX_val.shape)
print(BaseY_val.shape)
print(X_test.shape)
print(Y_test.shape)

(6474, 32, 32, 1)
(6629376,)
(6474, 4)
(1728, 32, 32, 1)
(1728, 4)
(175, 32, 32, 1)
(175, 4)


### PART 1: Fully connected layers 
--------------------------------------

---
<span style="color:red">***Task 1:***</span> *NN with fully connected layers. 

Elaborate a NN with 2 hidden fully connected layers with 300, 150 neurons and 4 neurons for classification. Use ReLU activation functions for the hidden layers and He_normal for initialization. Don't forget to flatten your image before feedforward to the first dense layer. Name the model `model_relu`.*

---

In [37]:
#--------------------------Impelment your code here:-------------------------------------
#Data Flattening:
BaseX_train_f = BaseX_train.flatten()
BaseY_train_f = BaseY_train.flatten()
BaseX_val_f = BaseX_val.flatten()
BaseY_val_f = BaseY_val.flatten()
X_test_f = X_test.flatten()
Y_test_f = Y_test.flatten()


n_filters_start = 300
n_filters_finish = 4
len_sub_window = 10
dropout = 0.2
he_normal_initializer = tf.keras.initializers.he_normal()
model_relu = Sequential()
model_relu.add(Flatten(input_shape = (32,32,1))) #input_shape defines the shape of the input: 32X32 is the image size, while 3 is the number of dimensions (RGB hence 3)
model_relu.add(Dense(n_filters_start, activation='relu',kernel_initializer = he_normal_initializer)) 
model_relu.add(Dropout(dropout))
model_relu.add(Dense(n_filters_start/2,activation='relu'))
model_relu.add(Dropout(dropout))
model_relu.add(Dense(n_filters_finish,activation='softmax'))


# another try model
# model_relu_d = Sequential(name="model_relu")
# model_relu_d.add(Dense(300, input_shape=(32 ** 2,), kernel_initializer="he_normal"))
# model_relu_d.add(Activation('relu', name='ReLU_1'))
# model_relu_d.add(Dropout(0.2))

# model_relu_d.add(Dense(150, kernel_initializer="he_normal"))
# model_relu_d.add(Activation('relu', name='ReLU_2'))
# model_relu_d.add(Dropout(0.2))

# model_relu_d.add(Dense(4))
# model_relu_d.add(Activation('softmax'))
#----------------------------------------------------------------------------------------

In [38]:
model_relu.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_33 (Dense)             (None, 300)               307500    
_________________________________________________________________
dropout_8 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 150)               45150     
_________________________________________________________________
dropout_9 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 4)                 604       
Total params: 353,254
Trainable params: 353,254
Non-trainable params: 0
________________________________________________

In [39]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)


Compile the model with the optimizer above, accuracy metric and adequate loss for multiclass task. Train your model on the training set and evaluate the model on the testing set. Print the accuracy and loss over the testing set.

In [40]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer = AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')
model_relu.fit(BaseX_train, BaseY_train, epochs = epochs, batch_size = batch_size, validation_data=(X_test,Y_test))

# This crossentropy loss function is used when there are two or more label classes. We expect labels to be provided in a one_hot representation. 
# If labels were  provided as integers, then SparseCategoricalCrossentropy loss should be used. There should be # classes floating point values per feature.
#----------------------------------------------------------------------------------------

Train on 6474 samples, validate on 175 samples
Epoch 1/25
6474/6474 [==============================] - 1s 166us/sample - loss: 1.3627 - accuracy: 0.3163 - val_loss: 1.2818 - val_accuracy: 0.4686
Epoch 2/25
6474/6474 [==============================] - 1s 84us/sample - loss: 1.2486 - accuracy: 0.4526 - val_loss: 1.2136 - val_accuracy: 0.5486
Epoch 3/25
6474/6474 [==============================] - 1s 85us/sample - loss: 1.1663 - accuracy: 0.5236 - val_loss: 1.1428 - val_accuracy: 0.6000
Epoch 4/25
6474/6474 [==============================] - 1s 90us/sample - loss: 1.1051 - accuracy: 0.5635 - val_loss: 1.1041 - val_accuracy: 0.6114
Epoch 5/25
6474/6474 [==============================] - 1s 82us/sample - loss: 1.0597 - accuracy: 0.5941 - val_loss: 1.0637 - val_accuracy: 0.6343
Epoch 6/25
6474/6474 [==============================] - 1s 88us/sample - loss: 1.0203 - accuracy: 0.6180 - val_loss: 1.0352 - val_accuracy: 0.6114
Epoch 7/25
6474/6474 [==============================] - 1s 83us/sample

In [41]:
model_relu.evaluate(X_test, Y_test)

175/175 [==============================] - 0s 114us/sample - loss: 0.8058 - accuracy: 0.6743


[0.8058029648235866, 0.6742857]

---
<span style="color:red">***Task 2:***</span> *Activation functions.* 

Change the activation functions to LeakyRelu or tanh or sigmoid. Name the new model `new_a_model`. Explain how it can affect the model.*

---

In [47]:
#--------------------------Impelment your code here:-------------------------------------
#import keras.layers.advanced_activations as advanced_activations
leaky_relu = LeakyReLU(alpha=0.1)


new_a_model = Sequential()
new_a_model.add(Flatten(input_shape = (32,32,1))) #input_shape defines the shape of the input: 32X32 is the image size, while 3 is the number of dimensions (RGB hence 3)
new_a_model.add(leaky_relu)
new_a_model.add(Dense(n_filters_start,kernel_initializer = he_normal_initializer)) 
new_a_model.add(leaky_relu)
new_a_model.add(Dropout(dropout))
new_a_model.add(Dense(n_filters_start/2))
new_a_model.add(leaky_relu)
new_a_model.add(Dropout(dropout))
new_a_model.add(Dense(n_filters_finish))
new_a_model.add(Dense(n_filters_finish,activation='softmax'))

new_a_model_25 = new_a_model
new_a_model_40 = new_a_model
#----------------------------------------------------------------------------------------

In [43]:
new_a_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 1024)              0         
_________________________________________________________________
leaky_re_lu (LeakyReLU)      multiple                  0         
_________________________________________________________________
dense_39 (Dense)             (None, 300)               307500    
_________________________________________________________________
dropout_12 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 150)               45150     
_________________________________________________________________
dropout_13 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 4)               

---
<span style="color:red">***Task 3:***</span> *Number of epochs.* 

Train the new model using 25 and 40 epochs. What difference does it makes in term of performance? Remember to save the compiled model for having initialized weights for every run as we did in tutorial 12. Evaluate each trained model on the test set*

---

In [50]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)



In [51]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model_25.compile(optimizer = AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')
new_a_model_25.fit(BaseX_train, BaseY_train, epochs = epochs, batch_size = batch_size, validation_data=(X_test,Y_test))
#-----------------------------------------------------------------------------------------

Train on 6474 samples, validate on 175 samples
Epoch 1/25
6474/6474 [==============================] - 1s 208us/sample - loss: 0.4434 - accuracy: 0.8550 - val_loss: 0.8951 - val_accuracy: 0.6514
Epoch 2/25
6474/6474 [==============================] - 1s 87us/sample - loss: 0.4432 - accuracy: 0.8551 - val_loss: 0.8767 - val_accuracy: 0.6571
Epoch 3/25
6474/6474 [==============================] - 1s 87us/sample - loss: 0.4420 - accuracy: 0.8562 - val_loss: 0.8842 - val_accuracy: 0.6514
Epoch 4/25
6474/6474 [==============================] - 1s 86us/sample - loss: 0.4377 - accuracy: 0.8585 - val_loss: 0.8858 - val_accuracy: 0.6514
Epoch 5/25
6474/6474 [==============================] - 1s 94us/sample - loss: 0.4343 - accuracy: 0.8610 - val_loss: 0.8938 - val_accuracy: 0.6514
Epoch 6/25
6474/6474 [==============================] - 1s 93us/sample - loss: 0.4311 - accuracy: 0.8636 - val_loss: 0.8899 - val_accuracy: 0.6571
Epoch 7/25
6474/6474 [==============================] - 1s 91us/sample

In [45]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 40

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)



In [49]:
#--------------------------Impelment your code here:-------------------------------------

new_a_model_40.compile(optimizer = AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')
new_a_model_40.fit(BaseX_train, BaseY_train, epochs = epochs, batch_size = batch_size, validation_data=(X_test,Y_test))

#-----------------------------------------------------------------------------------------

Train on 6474 samples, validate on 175 samples
Epoch 1/40
6474/6474 [==============================] - 1s 179us/sample - loss: 0.5675 - accuracy: 0.8156 - val_loss: 0.8161 - val_accuracy: 0.6743
Epoch 2/40
6474/6474 [==============================] - 1s 86us/sample - loss: 0.5596 - accuracy: 0.8182 - val_loss: 0.8211 - val_accuracy: 0.6686
Epoch 3/40
6474/6474 [==============================] - 1s 88us/sample - loss: 0.5545 - accuracy: 0.8173 - val_loss: 0.8168 - val_accuracy: 0.6857
Epoch 4/40
6474/6474 [==============================] - 1s 90us/sample - loss: 0.5521 - accuracy: 0.8182 - val_loss: 0.8135 - val_accuracy: 0.6743
Epoch 5/40
6474/6474 [==============================] - 1s 96us/sample - loss: 0.5504 - accuracy: 0.8233 - val_loss: 0.8220 - val_accuracy: 0.6743
Epoch 6/40
6474/6474 [==============================] - 1s 96us/sample - loss: 0.5478 - accuracy: 0.8225 - val_loss: 0.8344 - val_accuracy: 0.6629
Epoch 7/40
6474/6474 [==============================] - 1s 92us/sample

### Evaluation results:

In [54]:
new_a_model_25.evaluate(X_test, Y_test)

175/175 [==============================] - 0s 131us/sample - loss: 0.9243 - accuracy: 0.6457


[0.9242858869688851, 0.6457143]

In [55]:
new_a_model_40.evaluate(X_test, Y_test)

175/175 [==============================] - 0s 120us/sample - loss: 0.9243 - accuracy: 0.6457


[0.9242858869688851, 0.6457143]

### Saving the models:

In [56]:
if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"

#model_relu:
model_name = "model_relu.h5"
model_path = os.path.join(save_dir, model_name)
model_relu.save(model_path)
print('Saved trained model at %s ' % model_path)

#new_a_model - 25 epochs:
model_name = "new_a_model_25.h5"
model_path = os.path.join(save_dir, model_name)
new_a_model_25.save(model_path)
print('Saved trained model at %s ' % model_path)


#new_a_model - 40 epochs:
model_name = "new_a_model_40.h5"
model_path = os.path.join(save_dir, model_name)
new_a_model_40.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at results/model_relu.h5 
Saved trained model at results/new_a_model_25.h5 
Saved trained model at results/new_a_model_40.h5 


---
<span style="color:red">***Task 4:***</span> *Mini-batches.* 

Build the `model_relu` again and run it with a batch size of 32 instead of 64. What are the advantages of the mini-batch vs. SGD?*

---

In [57]:
keras.backend.clear_session()

In [58]:
#--------------------------Impelment your code here:-------------------------------------
# 64 batch-size:
model_relu = Sequential()
model_relu.add(Flatten(input_shape = (32,32,1))) #input_shape defines the shape of the input: 32X32 is the image size, while 3 is the number of dimensions (RGB hence 3)
model_relu.add(Dense(n_filters_start, activation='relu',kernel_initializer = he_normal_initializer)) 
model_relu.add(Dropout(dropout))
model_relu.add(Dense(n_filters_start/2,activation='relu'))
model_relu.add(Dropout(dropout))
model_relu.add(Dense(n_filters_finish,activation='softmax'))

#----------------------------------------------------------------------------------------

In [60]:
batch_size = 32
epochs = 50

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)


In [61]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer = AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')
model_relu.fit(BaseX_train, BaseY_train, epochs = epochs, batch_size = batch_size, validation_data=(X_test,Y_test))
#----------------------------------------------------------------------------------------

Train on 6474 samples, validate on 175 samples
Epoch 1/50
6474/6474 [==============================] - 2s 239us/sample - loss: 1.3167 - accuracy: 0.3665 - val_loss: 1.2095 - val_accuracy: 0.5829
Epoch 2/50
6474/6474 [==============================] - 1s 151us/sample - loss: 1.1841 - accuracy: 0.5017 - val_loss: 1.1213 - val_accuracy: 0.6229
Epoch 3/50
6474/6474 [==============================] - 1s 153us/sample - loss: 1.1023 - accuracy: 0.5650 - val_loss: 1.0568 - val_accuracy: 0.6343
Epoch 4/50
6474/6474 [==============================] - 1s 146us/sample - loss: 1.0406 - accuracy: 0.6129 - val_loss: 1.0129 - val_accuracy: 0.6457
Epoch 5/50
6474/6474 [==============================] - 1s 147us/sample - loss: 0.9877 - accuracy: 0.6358 - val_loss: 0.9767 - val_accuracy: 0.6343
Epoch 6/50
6474/6474 [==============================] - 1s 146us/sample - loss: 0.9464 - accuracy: 0.6594 - val_loss: 0.9445 - val_accuracy: 0.6457
Epoch 7/50
6474/6474 [==============================] - 1s 157us/

In [62]:
model_relu.evaluate(X_test, Y_test)

175/175 [==============================] - 0s 130us/sample - loss: 0.8532 - accuracy: 0.6800


[0.8531896362985883, 0.68]

---
<span style="color:red">***Task 4:***</span> *Batch normalization.* 

Build the `new_a_model` again and add batch normalization layers. How does it impact your results?*

---

In [63]:
keras.backend.clear_session()

In [65]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model_batch = Sequential()
new_a_model_batch.add(Flatten(input_shape = (32,32,1))) #input_shape defines the shape of the input: 32X32 is the image size, while 3 is the number of dimensions (RGB hence 3)
new_a_model_batch.add(leaky_relu)
new_a_model_batch.add(BatchNormalization())
new_a_model_batch.add(Dense(n_filters_start,kernel_initializer = he_normal_initializer)) 
new_a_model_batch.add(leaky_relu)
new_a_model_batch.add(BatchNormalization())
new_a_model_batch.add(Dropout(dropout))
new_a_model_batch.add(BatchNormalization())
new_a_model_batch.add(Dense(n_filters_start/2))
new_a_model_batch.add(leaky_relu)
new_a_model_batch.add(BatchNormalization())
new_a_model_batch.add(Dropout(dropout))
new_a_model_batch.add(BatchNormalization())
new_a_model_batch.add(Dense(n_filters_finish))
new_a_model_batch.add(BatchNormalization())
new_a_model_batch.add(Dense(n_filters_finish,activation='softmax'))


#---------------------------------------------------------------------------------------

In [66]:
batch_size = 32
epochs = 50

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)
#Compile the network: 




In [67]:
#Preforming the training by using fit 
#--------------------------Impelment your code here:-------------------------------------
new_a_model_batch.compile(optimizer = AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')
new_a_model_batch.fit(BaseX_train, BaseY_train, epochs = epochs, batch_size = batch_size, validation_data=(X_test,Y_test))
#----------------------------------------------------------------------------------------

Train on 6474 samples, validate on 175 samples
Epoch 1/50
6474/6474 [==============================] - 4s 646us/sample - loss: 1.2095 - accuracy: 0.4583 - val_loss: 1.2938 - val_accuracy: 0.3657
Epoch 2/50
6474/6474 [==============================] - 2s 255us/sample - loss: 0.9847 - accuracy: 0.6091 - val_loss: 1.1088 - val_accuracy: 0.5429
Epoch 3/50
6474/6474 [==============================] - 2s 264us/sample - loss: 0.8827 - accuracy: 0.6823 - val_loss: 1.0235 - val_accuracy: 0.6457
Epoch 4/50
6474/6474 [==============================] - 2s 263us/sample - loss: 0.8324 - accuracy: 0.7212 - val_loss: 0.9809 - val_accuracy: 0.6571
Epoch 5/50
6474/6474 [==============================] - 2s 266us/sample - loss: 0.8051 - accuracy: 0.7476 - val_loss: 0.9534 - val_accuracy: 0.6857
Epoch 6/50
6474/6474 [==============================] - 2s 267us/sample - loss: 0.7808 - accuracy: 0.7680 - val_loss: 0.9356 - val_accuracy: 0.7086
Epoch 7/50
6474/6474 [==============================] - 2s 266us/

In [72]:
#model_relu - batch = 32:
model_name = "model_relu_32.h5"
model_path = os.path.join(save_dir, model_name)
model_relu.save(model_path)
print('Saved trained model at %s ' % model_path)

#new_a_model - with batch normalization:
model_name = "new_a_model_batch_norm.h5"
model_path = os.path.join(save_dir, model_name)
new_a_model_batch.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at results/model_relu_32.h5 
Saved trained model at results/new_a_model_batch_norm.h5 


### PART 2: Convolutional Neural Network (CNN)
------------------------------------------------------------------------------------

---
<span style="color:red">***Task 1:***</span> *2D CNN.* 

Have a look at the model below and answer the following:
* How many layers does it have?
* How many filter in each layer?
* Would the number of parmaters be similar to a fully connected NN?
* Is this specific NN performing regularization?

---

In [68]:
def get_net(input_shape,drop,dropRate,reg):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = input_shape))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_1',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_2',kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_4',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_5',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model

In [69]:
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 1e-03
batch_size = 64
epochs = 25
drop = True
dropRate = 0.3
reg = 1e-2
NNet = get_net(input_shape,drop,dropRate,reg)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
dropout_4 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 64)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 128)      

In [70]:
NNet=get_net(input_shape,drop,dropRate,reg)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
dropout_10 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 32, 32, 64)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_11 (Dropout)         (None, 16, 16, 128)      

In [73]:
from tensorflow.keras.optimizers import *
import os
from tensorflow.keras.callbacks import *

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

#Compile the network: 
NNet.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')

#Saving checkpoints during training:
Checkpath = os.getcwd()
Checkp = ModelCheckpoint(Checkpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, save_freq=1)

In [74]:
#Preforming the training by using fit 
# IMPORTANT NOTE: This will take a few minutes!
h = NNet.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
#NNet.save(model_fn)

Train on 6474 samples, validate on 1728 samples
Epoch 1/25
6474/6474 [==============================] - 121s 19ms/sample - loss: 8.1043 - acc: 0.4272 - val_loss: 7.8786 - val_acc: 0.2500
Epoch 2/25
6474/6474 [==============================] - 118s 18ms/sample - loss: 7.6432 - acc: 0.5310 - val_loss: 7.9201 - val_acc: 0.2500
Epoch 3/25
6474/6474 [==============================] - 118s 18ms/sample - loss: 7.4605 - acc: 0.5884 - val_loss: 8.0081 - val_acc: 0.2517
Epoch 4/25
6474/6474 [==============================] - 119s 18ms/sample - loss: 7.3379 - acc: 0.6118 - val_loss: 8.0063 - val_acc: 0.2575
Epoch 5/25
6474/6474 [==============================] - 120s 18ms/sample - loss: 7.2537 - acc: 0.6359 - val_loss: 7.8991 - val_acc: 0.2708
Epoch 6/25
6474/6474 [==============================] - 119s 18ms/sample - loss: 7.1866 - acc: 0.6555 - val_loss: 7.7292 - val_acc: 0.3229
Epoch 7/25
6474/6474 [==============================] - 120s 19ms/sample - loss: 7.1363 - acc: 0.6670 - val_loss: 7.66

In [ ]:
# NNet.load_weights('Weights_1.h5')

In [76]:
#new_a_model - with batch normalization:
model_name = "NNet.h5"
model_path = os.path.join(save_dir, model_name)
NNet.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at results/NNet.h5 


In [75]:
results = NNet.evaluate(X_test,Y_test)
print('test loss, test acc:', results)

175/175 [==============================] - 1s 8ms/sample - loss: 7.9831 - acc: 0.2971
test loss, test acc: [7.983084681374686, 0.29714286]


---
<span style="color:red">***Task 2:***</span> *Number of filters* 

Rebuild the function `get_net` to have as an input argument a list of number of filters in each layers, i.e. for the CNN defined above the input should have been `[64, 128, 128, 256, 256]`. Now train the model with the number of filters reduced by half. What were the results.

---

In [78]:
#--------------------------Impelment your code here:-------------------------------------
def get_net(input_shape,drop,dropRate,reg, filter_num1, filter_num2, filter_num3, filter_num4, filter_num5):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = input_shape))
    model.add(Conv2D(filters=filter_num1, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_1',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=filter_num2, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_2',kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=filter_num3, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=filter_num4, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_4',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=filter_num5, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_5',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model
#----------------------------------------------------------------------------------------

In [79]:
filter_n1 = 32
filter_n2 = 64
filter_n3 = 64
filter_n4 = 128
filter_n5 = 128

NNet_half = get_net(input_shape,drop,dropRate,reg, filter_n1, filter_n2, filter_n3, filter_n4, filter_n5)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_2 (Permute)          (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
dropout_16 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_17 (Dropout)         (None, 16, 16, 64)       

In [81]:
#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

#Compile the network: 
NNet_half.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')

#Saving checkpoints during training:
Checkpath = os.getcwd()
Checkp = ModelCheckpoint(Checkpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, save_freq=1)

#Preforming the training by using fit 
# IMPORTANT NOTE: This will take a few minutes!
h = NNet_half.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
#NNet_half.save(model_fn)

Train on 6474 samples, validate on 1728 samples
Epoch 1/25
6474/6474 [==============================] - 55s 9ms/sample - loss: 5.0151 - acc: 0.3593 - val_loss: 4.6334 - val_acc: 0.2500
Epoch 2/25
6474/6474 [==============================] - 53s 8ms/sample - loss: 4.5919 - acc: 0.4804 - val_loss: 4.7241 - val_acc: 0.2500
Epoch 3/25
6474/6474 [==============================] - 47s 7ms/sample - loss: 4.4434 - acc: 0.5256 - val_loss: 4.7973 - val_acc: 0.2500
Epoch 4/25
6474/6474 [==============================] - 49s 8ms/sample - loss: 4.3581 - acc: 0.5462 - val_loss: 4.7466 - val_acc: 0.2668
Epoch 5/25
6474/6474 [==============================] - 46s 7ms/sample - loss: 4.2522 - acc: 0.5783 - val_loss: 4.6592 - val_acc: 0.2963
Epoch 6/25
6474/6474 [==============================] - 46s 7ms/sample - loss: 4.2161 - acc: 0.5853 - val_loss: 4.5386 - val_acc: 0.3461
Epoch 7/25
6474/6474 [==============================] - 46s 7ms/sample - loss: 4.1602 - acc: 0.6109 - val_loss: 4.4724 - val_acc: 

In [82]:
#new_a_model - with batch normalization:
model_name = "NNet_half.h5"
model_path = os.path.join(save_dir, model_name)
NNet_half.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at results/NNet_half.h5 


In [83]:
results = NNet_half.evaluate(X_test,Y_test)
print('test loss, test acc:', results)

175/175 [==============================] - 1s 3ms/sample - loss: 4.6617 - acc: 0.3714
test loss, test acc: [4.661678107125419, 0.37142858]


That's all folks! See you :)